In [1]:
import pyspark
from pyspark.sql import SparkSession
import pandas as pd
from pyspark.sql import types

In [2]:

spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/02/25 17:33:08 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [ ]:
df = spark.read \
    .option("header", "true") \
    .csv('taxi+_zone_lookup.csv')

df.show()

In [ ]:
spark.version

In [ ]:
!wget https://github.com/DataTalksClub/nyc-tlc-data/releases/download/fhvhv/fhvhv_tripdata_2021-06.csv.gz

In [28]:
fhv_schema = types.StructType([
    types.StructField('dispatching_base_num', types.StringType(), True),
    types.StructField('pickup_datetime', types.TimestampType(), True),
    types.StructField('dropoff_datetime', types.TimestampType(), True),
    types.StructField('PULocationID', types.StringType(), True),
    types.StructField('SR_Flag', types.IntegerType(), True),
    types.StructField('DOLocationID', types.IntegerType(), True),
    types.StructField('hvfhs_license_num', types.StringType(), True),
])

In [29]:
!pwd

/home/antonis/notebooks


In [30]:
df_fhv = spark.read \
        .option("header", "true") \
        .schema(fhv_schema) \
        .csv('fhvhv_tripdata_2021-06.csv.gz')

In [9]:
df_fhv.show()

23/02/25 17:42:25 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: dispatching_base_num, pickup_datetime, dropoff_datetime, PULocationID, DOLocationID, SR_Flag, Affiliated_base_number
 Schema: dispatching_base_num, pickup_datetime, dropoff_datetime, SR_Flag, PULocationID, DOLocationID, hvfhs_license_num
Expected: SR_Flag but found: PULocationID
CSV file: file:///home/antonis/notebooks/fhvhv_tripdata_2021-06.csv.gz
+--------------------+-------------------+-------------------+-------+------------+------------+-----------------+
|dispatching_base_num|    pickup_datetime|   dropoff_datetime|SR_Flag|PULocationID|DOLocationID|hvfhs_license_num|
+--------------------+-------------------+-------------------+-------+------------+------------+-----------------+
|              B02764|2021-06-01 00:02:41|2021-06-01 00:07:46|    174|          18|        null|           B02764|
|              B02764|2021-06-01 00:16:16|2021-06-01 00:21:14|     32|         254|        null|

In [5]:
output_path = 'data/fhv/parquet'

df_fhv = spark.read \
        .option("header", "true") \
        .schema(fhv_schema) \
        .csv('fhvhv_tripdata_2021-06.csv.gz')

df_fhv \
    .repartition(12) \
    .write.parquet(output_path)

23/02/25 17:37:02 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: dispatching_base_num, pickup_datetime, dropoff_datetime, PULocationID, DOLocationID, SR_Flag, Affiliated_base_number
 Schema: dispatching_base_num, pickup_datetime, dropoff_datetime, SR_Flag, PULocationID, DOLocationID, hvfhs_license_num
Expected: SR_Flag but found: PULocationID
CSV file: file:///home/antonis/notebooks/fhvhv_tripdata_2021-06.csv.gz


In [ ]:
df_fhv.show()

In [31]:
df_fhv.registerTempTable('trips_data')

/home/antonis/spark/spark-3.3.2-bin-hadoop3/python/pyspark/sql/dataframe.py:229: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  warnings.warn("Deprecated in 2.0, use createOrReplaceTempView instead.", FutureWarning)


In [10]:
# Q3
spark.sql("""
SELECT
COUNT(*)
FROM
    trips_data
    WHERE pickup_datetime >= '2021-06-15 00:00:00' AND pickup_datetime <= '2021-06-15 23:59:00'
""").show()

+--------+
|count(1)|
+--------+
|  452133|
+--------+



In [18]:
# Q4
spark.sql("""

SELECT MAX(diff)
FROM
(
SELECT
(bigint(to_timestamp(dropoff_datetime))) - (bigint(to_timestamp(pickup_datetime))) as diff
FROM
    trips_data
)
""").show()

+---------+
|max(diff)|
+---------+
|   240764|
+---------+



In [32]:
# Q6
spark.sql("""
SELECT
PULocationID,
COUNT(*) AS total
FROM
    trips_data
GROUP BY
PULocationID
ORDER BY total DESC
LIMIT 5 
""").show()

+------------+------+
|PULocationID| total|
+------------+------+
|          61|231279|
|          79|221244|
|         132|188867|
|          37|187929|
|          76|186780|
+------------+------+



In [26]:
zones = pd.read_csv('taxi+_zone_lookup.csv')
zones.head(5)

,LocationID,Borough,Zone,service_zone
0,1,EWR,Newark Airport,EWR
1,2,Queens,Jamaica Bay,Boro Zone
2,3,Bronx,Allerton/Pelham Gardens,Boro Zone
3,4,Manhattan,Alphabet City,Yellow Zone
4,5,Staten Island,Arden Heights,Boro Zone


In [33]:
zones[zones['LocationID'] == 61]

,LocationID,Borough,Zone,service_zone
60,61,Brooklyn,Crown Heights North,Boro Zone
